In [19]:
# Use external ConCare architecture from the uploaded file
import sys, os
sys.path.append('/mnt/data')  # Ensure the uploaded model file is importable
from ConCare_Model_v2 import ConCare  # noqa: F401
print('Imported ConCare from ConCare_Model_v2.py successfully')


Imported ConCare from ConCare_Model_v2.py successfully


In [2]:
# Quick sanity check of ConCare signature
try:
    _sig = ConCare.__init__.__code__.co_varnames
    print('ConCare init args:', _sig)
except Exception as e:
    print('Warning reading ConCare signature:', e)


ConCare init args: ('self', 'input_dim', 'hidden_dim', 'd_model', 'MHD_num_head', 'd_ff', 'output_dim', 'keep_prob', 'demographic_dim')


In [3]:
import numpy as np
import argparse
import os
import importlib
import re
import pickle
import datetime
import random
import math
import copy


import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F


from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

In [4]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)

Using device: cuda


### Prepare

In [6]:
data_path = './data/'
file_name = './model/concare_v1'
small_part = False
arg_timestep = 1.0
batch_size = 256
epochs = 100

In [7]:
# Build readers, discretizers, normalizers
train_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'train'),
                                         listfile=os.path.join(data_path, 'train_listfile.csv'),
                                         period_length=48.0)

val_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'train'),
                                       listfile=os.path.join(data_path, 'val_listfile.csv'),
                                       period_length=48.0)

discretizer = Discretizer(timestep=arg_timestep,
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

In [8]:
discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
normalizer_state = 'ihm_normalizer'
normalizer_state = os.path.join(os.path.dirname(data_path), normalizer_state)
normalizer.load_params(normalizer_state)

In [9]:
n_trained_chunks = 0
train_raw = utils.load_data(train_reader, discretizer, normalizer, small_part, return_names=True)
val_raw = utils.load_data(val_reader, discretizer, normalizer, small_part, return_names=True)

In [10]:
demographic_data = []
diagnosis_data = []
idx_list = []

demo_path = data_path + 'demographic/'
for cur_name in os.listdir(demo_path):
    cur_id, cur_episode = cur_name.split('_', 1)
    cur_episode = cur_episode[:-4]
    cur_file = demo_path + cur_name

    with open(cur_file, "r") as tsfile:
        header = tsfile.readline().strip().split(',')
        if header[0] != "Icustay":
            continue
        cur_data = tsfile.readline().strip().split(',')
        
    if len(cur_data) == 1:
        cur_demo = np.zeros(12)
        cur_diag = np.zeros(128)
    else:
        if cur_data[3] == '':
            cur_data[3] = 60.0
        if cur_data[4] == '':
            cur_data[4] = 160
        if cur_data[5] == '':
            cur_data[5] = 60

        cur_demo = np.zeros(12)
        cur_demo[int(cur_data[1])] = 1
        cur_demo[5 + int(cur_data[2])] = 1
        cur_demo[9:] = cur_data[3:6]
        cur_diag = np.array(cur_data[8:], dtype=int)

    demographic_data.append(cur_demo)
    diagnosis_data.append(cur_diag)
    idx_list.append(cur_id+'_'+cur_episode)

for each_idx in range(9,12):
    cur_val = []
    for i in range(len(demographic_data)):
        cur_val.append(demographic_data[i][each_idx])
    cur_val = np.array(cur_val)
    _mean = np.mean(cur_val)
    _std = np.std(cur_val)
    _std = _std if _std > 1e-7 else 1e-7
    for i in range(len(demographic_data)):
        demographic_data[i][each_idx] = (demographic_data[i][each_idx] - _mean) / _std

### model

In [11]:
def get_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred, y_true)

In [12]:
class Dataset(data.Dataset):
    def __init__(self, x, y, name):
        self.x = x
        self.y = y
        self.name = name

    def __getitem__(self, index):#返回的是tensor
        return self.x[index], self.y[index], self.name[index]

    def __len__(self):
        return len(self.x)

In [13]:
train_dataset = Dataset(train_raw['data'][0], train_raw['data'][1], train_raw['names'])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataset = Dataset(val_raw['data'][0], val_raw['data'][1], val_raw['names'])
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

### Run

In [14]:
try:
    DEM_DIM = len(demographic_data[0])
except Exception:
    DEM_DIM = 12  # fallback; adjust if your demographics vector has a different size

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  # numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)  # cpu
torch.cuda.manual_seed(RANDOM_SEED)  # gpu
torch.backends.cudnn.deterministic = True  # cudnn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = ConCare(input_dim=76, hidden_dim=64, d_model=64, MHD_num_head=4, d_ff=256, output_dim=1, keep_prob=0.8, demographic_dim=DEM_DIM).to(device)
# input_dim, d_model, d_k, d_v, MHD_num_head, d_ff, output_dim
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

max_roc = 0
max_prc = 0
train_loss = []
train_model_loss = []
train_decov_loss = []
valid_loss = []
valid_model_loss = []
valid_decov_loss = []
history = []
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

# Build once for O(1) demographic lookup
demo_map = {idx_list[i]: torch.tensor(demographic_data[i], dtype=torch.float32) for i in range(len(idx_list))}

for each_epoch in range(100):
    batch_loss = []
    model_batch_loss = []
    decov_batch_loss = []

    model.train()

    for step, (batch_x, batch_y, batch_name) in enumerate(train_loader):
        optimizer.zero_grad()
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)

        # Fast demographic batch build
        batch_demo = [demo_map[f"{batch_name[i].split('_', 2)[0]}_{batch_name[i].split('_', 2)[1]}"] for i in range(len(batch_name))]
        batch_demo = torch.stack(batch_demo).to(device)

        output, decov_loss = model(batch_x, batch_demo)

        model_loss = get_loss(output, batch_y.unsqueeze(-1))
        loss = model_loss + 800 * decov_loss

        batch_loss.append(loss.item())
        model_batch_loss.append(model_loss.item())
        decov_batch_loss.append(decov_loss.item())
        loss.backward()
        optimizer.step()

        if step % 30 == 0:
            print('Epoch %d Batch %d: Train Loss = %.4f' % (each_epoch, step, np.mean(np.array(batch_loss))))
            print('Model Loss = %.4f, Decov Loss = %.4f' % (np.mean(np.array(model_batch_loss)), np.mean(np.array(decov_batch_loss))))
    train_loss.append(np.mean(np.array(batch_loss)))
    train_model_loss.append(np.mean(np.array(model_batch_loss)))
    train_decov_loss.append(np.mean(np.array(decov_batch_loss)))

    batch_loss = []
    model_batch_loss = []
    decov_batch_loss = []

    y_true = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        for step, (batch_x, batch_y, batch_name) in enumerate(valid_loader):
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)

            # Fast demographic batch build
            batch_demo = [demo_map[f"{batch_name[i].split('_', 2)[0]}_{batch_name[i].split('_', 2)[1]}"] for i in range(len(batch_name))]
            batch_demo = torch.stack(batch_demo).to(device)

            output, decov_loss = model(batch_x, batch_demo)

            model_loss = get_loss(output, batch_y.unsqueeze(-1))

            loss = model_loss + 10 * decov_loss
            batch_loss.append(loss.item())
            model_batch_loss.append(model_loss.item())
            decov_batch_loss.append(decov_loss.item())
            y_pred += list(output.cpu().detach().numpy().flatten())
            y_true += list(batch_y.cpu().numpy().flatten())

    valid_loss.append(np.mean(np.array(batch_loss)))
    valid_model_loss.append(np.mean(np.array(model_batch_loss)))
    valid_decov_loss.append(np.mean(np.array(decov_batch_loss)))

    print("\n==>Predicting on validation")
    print('Valid Loss = %.4f' % (valid_loss[-1]))
    print('valid_model Loss = %.4f' % (valid_model_loss[-1]))
    print('valid_decov Loss = %.4f' % (valid_decov_loss[-1]))
    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    ret = metrics.print_metrics_binary(y_true, y_pred)
    history.append(ret)
    print()

    cur_auroc = ret['auroc']

    if cur_auroc > max_roc:
        max_roc = cur_auroc
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': each_epoch
        }
        # Ensure parent directory exists before saving
        import os
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        torch.save(state, file_name)
        print('\n------------ Save best model ------------\n')

Using device: cuda
Epoch 0 Batch 0: Train Loss = 0.7042
Model Loss = 0.7026, Decov Loss = 0.0000
Epoch 0 Batch 30: Train Loss = 0.4978
Model Loss = 0.4947, Decov Loss = 0.0000

==>Predicting on validation
Valid Loss = 0.3906
valid_model Loss = 0.3906
valid_decov Loss = 0.0000
confusion matrix:
[[2786    0]
 [ 436    0]]
accuracy = 0.8646803227808815
precision class 0 = 0.8646803227808815
precision class 1 = 0.0
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.6441356520479198
AUC of PRC = 0.2092614488638167
min(+P, Se) = 0.2356687898089172
f1_score = 0.0


------------ Save best model ------------

Epoch 1 Batch 0: Train Loss = 0.4358
Model Loss = 0.4353, Decov Loss = 0.0000
Epoch 1 Batch 30: Train Loss = 0.3906
Model Loss = 0.3900, Decov Loss = 0.0000

==>Predicting on validation
Valid Loss = 0.3534
valid_model Loss = 0.3534
valid_decov Loss = 0.0000
confusion matrix:
[[2784    2]
 [ 436    0]]
accuracy = 0.8640595903165735
precision class 0 = 0.8645962732919255
precision clas

### Run for test

In [15]:
checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'test'),
                                            listfile=os.path.join(data_path, 'test_listfile.csv'),
                                            period_length=48.0)
test_raw = utils.load_data(test_reader, discretizer, normalizer, small_part, return_names=True)
test_dataset = Dataset(test_raw['data'][0], test_raw['data'][1], test_raw['names'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

C:\Users\alper\AppData\Local\Temp\ipykernel_77568\3123361523.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(file_name)


In [16]:
batch_loss = []
y_true = []
y_pred = []

with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(test_loader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)

        # O(1) demographic lookup using the same key scheme as train/valid
        batch_demo = [demo_map[f"{bn.split('_', 2)[0]}_{bn.split('_', 2)[1]}"] for bn in batch_name]
        batch_demo = torch.stack(batch_demo).to(device)

        # If model returns (logits, decov_loss)
        output, _ = model(batch_x, batch_demo)

        loss = get_loss(output, batch_y.unsqueeze(-1))
        batch_loss.append(loss.item())

        # If get_loss uses BCEWithLogitsLoss, convert logits -> probs for metrics
        probs = output.detach().cpu().numpy().flatten()
        y_pred.extend(probs)
        y_true.extend(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f' % (np.mean(np.array(batch_loss))))

y_pred = np.array(y_pred)                                   # positive-class probs in [0, 1]
y_pred_prob = np.stack([1.0 - y_pred, y_pred], axis=1)      # shape (N, 2) = [P0, P1]
test_res = metrics.print_metrics_binary(y_true, y_pred_prob)






==>Predicting on test
Test Loss = 0.2768
confusion matrix:
[[2784   78]
 [ 254  120]]
accuracy = 0.8974042027194067
precision class 0 = 0.9163923633969717
precision class 1 = 0.6060606060606061
recall class 0 = 0.9727463312368972
recall class 1 = 0.32085561497326204
AUC of ROC = 0.8480980728483503
AUC of PRC = 0.4428800853605693
min(+P, Se) = 0.4732620320855615
f1_score = 0.4195804195804196


In [17]:
# Bootstrap evaluation
y_true_arr = np.asarray(y_true, dtype=int)
y_pred_arr = np.asarray(y_pred_prob, dtype=float)   # 1D positive-class probabilities

N = len(y_true_arr)
N_idx = np.arange(N)
K = 1000

auroc = []
auprc = []
minpse = []
for i in range(K):
    boot_idx = np.random.choice(N_idx, N, replace=True)
    boot_true = y_true_arr[boot_idx]
    boot_pred = y_pred_arr[boot_idx, :]  # 1D slice is correct

    test_ret = metrics.print_metrics_binary(boot_true, boot_pred, verbose=0)
    auroc.append(test_ret["auroc"])
    auprc.append(test_ret["auprc"])
    minpse.append(test_ret["minpse"])

    if (i + 1) % 50 == 0:
        print(f"{i+1}/{K}")

print("auroc %.4f(%.4f)" % (np.mean(auroc), np.std(auroc)))
print("auprc %.4f(%.4f)" % (np.mean(auprc), np.std(auprc)))
print("minpse %.4f(%.4f)" % (np.mean(minpse), np.std(minpse)))

50/1000
100/1000
150/1000
200/1000
250/1000
300/1000
350/1000
400/1000
450/1000
500/1000
550/1000
600/1000
650/1000
700/1000
750/1000
800/1000
850/1000
900/1000
950/1000
1000/1000
auroc 0.8484(0.0098)
auprc 0.4451(0.0276)
minpse 0.4748(0.0225)
